## Entrenamiento (Google Colab)

### Montar Google Drive para persistir los resultados

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Validar si se esta usando GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 24 18:42:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    47W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Clonar repositorio e instalar dependencias

In [ ]:
RUTA_PROYECTO_DRIVE = '/content/gdrive/MyDrive/Detector_Baches/Proyecto_Tesis'
YOLO = 'yolov5'

In [ ]:

# Clonar repositorio de tesis
!git clone https://github.com/allanes/detector_baches.git
%mv detector_baches/* $RUTA_PROYECTO_DRIVE/

%cd $RUTA_PROYECTO_DRIVE/$YOLO
%pip install -r requirements.txt  # install

In [4]:
import torch

if YOLO == 'yolov5':
    import utils
    display = utils.notebook_init()  # checks

INFO:yolov5:YOLOv5 🚀 2022-10-15 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)
YOLOv5 🚀 2022-10-15 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 23.0/78.2 GB disk)


### Descargar dataset de roboflow

In [ ]:
CARPETA_DATASET = 'Detector_baches-4'

In [ ]:
# Descargar y extraer el dataset de roboflow (se descarga en la carpeta de Yolo)
%cd $RUTA_PROYECTO_DRIVE/$YOLO
%mkdir $CARPETA_DATASET
%cd $CARPETA_DATASET
!curl -L "https://app.roboflow.com/ds/EE7FTxJYLN?key=MkOErKpnbW" > roboflow.zip
!unzip roboflow.zip
%rm roboflow.zip
%cd $RUTA_PROYECTO_DRIVE/

### Logging de datos

Usar el siguiente bloque si se quiere loguear el entrenamiento en alguna plataforma (requiere usuario)

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['TensorBoard', 'Comet', 'ClearML', 'wandb']

if logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train
elif logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml && clearml-init
elif logger == 'Weights and Biases':
  %pip install wandb && wandb login
  import wandb
  wandb.init()


### Entrenamiento

NOTA: Setear las siguientes constantes según corresponda

In [ ]:
YOLO_CONFIG = 'custom_cfg/custom_cfg_yolov5l.yaml'
YOLO_PESOS = 'yolov5l-cls.pt'
YOLO_HYP = 'custom_cfg/custom_hyps_yolov5l.yaml'

EPOCAS = 100

In [13]:
%%time

!python $YOLO/train.py --img 416 --batch 128 --epochs $EPOCAS --data $YOLO/$CARPETA_DATASET/data.yaml --cfg $YOLO_CONFIG --weights $YOLO_PESOS --hyp $YOLO_HYP

train: weights=yolov5l-cls.pt, cfg=custom_cfg/custom_hyps_yolov5l.yaml, data=yolov5/Detector_baches-4/data.yaml, hyp=custom_cfg/custom_augm_yolov5.yaml, epochs=100, batch_size=128, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2022-10-15 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.